## 원본 데이터로부터 모델 학습에 사용할 수 있는 데이터를 추출해보자.

프로세싱 순서는 아래와 같다.

1. 사용할 수 없는 데이터는 제외한 다음 상위 7개의 state를 가지는 음성만을 가져온다. 

2. sample rate를 16000으로 통일한다.

3. 파일의 이름을 통해 state를 설정하기 위해 파일 이름을 모두 변경한다.

4. 음성 파일 앞뒤의 화이트 노이즈(또는 특정 임계값 이하의 에너지를 가지는 시점)를 제거한다.

5. 음성이 모두 동일한 길이를 가지도록 조정한다.

6. 음성의 노이즈를 제거한다.

7. 모델의 학습 데이터로 사용한다.

> 4번 과정 대신 get_sample_data를 통해 임의로 n개의 파일을 선택해 학습을 수행하는 방법 또한 있다.

In [1]:
# Load packages
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# Set project path : 본인의 프로젝트가 있는 절대 경로를 append를 통해 추가한다.
main_path = '/Users/jaewone/developer/tensorflow/baby-cry-classification'

sys.path.append(main_path)
data_path = os.path.join(main_path, 'data')
sample_data_path = os.path.join(data_path, 'sample_data')
csv_path = os.path.join(main_path, 'info.csv')

In [3]:
# Load custon packages
from utils.os import *
from utils.sound import *
from trans_data.bit_sampling import *
from trans_data.get_state_list import get_state_list_from_dir_name
from trans_data.create_state_folder import create_state_folder
from trans_data.rename_by_state import rename_files_by_state
from trans_data.trim_white_noise import trim_audio

In [4]:
# 1. 사용할 수 없는 데이터는 제외한 다음 상위 7개의 state를 가지는 음성만을 가져온다. 
# 2. sample rate를 16000으로 통일한다.
create_state_folder(main_path, csv_path)

In [5]:
# 3. 파일의 이름을 통해 state를 설정하기 위해 파일 이름을 모두 변경한다.
file_list = rename_files_by_state(data_path)

In [6]:
# 4. 음성 파일 앞뒤의 화이트 노이즈(또는 특정 임계값 이하의 에너지를 가지는 시점)를 제거한다.
failed_file_list = []
for i in tqdm(range(len(file_list))):
    try:
        trim_audio(file_list[i], inplace=True)
    except:
        failed_file_list.append(file_list[i])

100%|██████████| 3472/3472 [24:32<00:00,  2.36it/s]


In [13]:
print(f'trimming에 실패한 {len(failed_file_list)}개의 파일이 삭제됨.')
for file in failed_file_list:
    file_list.remove(file)
    remove_file(file)

trimming에 실패한 6개의 파일이 삭제됨.


In [42]:
# 5. 음성이 모두 동일한 길이를 가지도록 조정한다.
df = pd.DataFrame({'file':file_list})
df['duration'] = df['file'].apply(lambda file: get_duration(file))
df['state'] = df['file'].apply(lambda x: x.rsplit('/', 1)[1].rsplit('_', 1)[0])
df['state'] = df['state'].astype('category')
df.tail(3)

,file,duration,state
3463,/Users/jaewone/developer/tensorflow/baby-cry-c...,13.684688,uncomfortable
3464,/Users/jaewone/developer/tensorflow/baby-cry-c...,15.766375,uncomfortable
3465,/Users/jaewone/developer/tensorflow/baby-cry-c...,17.136375,uncomfortable


In [47]:
df['state'].value_counts().keys()[:10]

CategoricalIndex(['sad', 'hungry', 'hug', 'awake', 'sleepy', 'uncomfortable',
                  'diaper', '03-02-06-01-30-150-04-01-00-16.wav',
                  '03-02-06-01-32-150-04-01-00-16.wav',
                  '10A40438-09AA-4A21-83B4-8119F03F7A11-1430925142-1.0-f-26-dc.wav'],
                 categories=['03-01-01-01-04-200-05-02-01-11.wav', '03-01-01-01-19-200-05-02-01-12.wav', '03-01-01-01-20-200-05-02-01-12.wav', '03-01-02-01-01-150-04-01-00-12.wav', '03-01-02-01-01-200-05-02-01-12.wav', '03-01-02-01-02-200-05-02-01-12.wav', '03-01-02-01-04-200-05-02-01-12.wav', '03-01-02-01-06-200-02-02-02-02.wav', ...], ordered=False, dtype='category')

In [ ]:
remove_file(csv_path)